In [2]:
# ==================== 1. Imports ====================
import os, gc
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
from PIL import Image

# ==================== 2. Config ====================
CONFIG = {
    "train_pkl":         "train.pkl",  # must exist in working dir
    "epochs":            50,
    "batch_size":        64,
    "resize":            (96, 96),
    "random_seed":       42,
    "val_split_ratio":   0.2,
    "num_workers":       2,
    "pin_memory":        True,
    "lr_backbone":       1e-4,
    "lr_classifier":     2e-4,
    "weight_decay":      1e-4,
    "scheduler_T_0":     10,
    "scheduler_T_mult":  2,
    "scheduler_eta_min": 1e-6,
    "dropout_1":         0.3,
    "dropout_2":         0.15,
}

# ==================== 3. Dataset ====================
class RPSDataset(Dataset):
    def __init__(self, X1, X2, y, resize, is_train):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.resize = resize
        self.is_train = is_train

        self.normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        self.to_tensor = transforms.ToTensor()
        self.augment = transforms.Compose([
            transforms.RandomResizedCrop(resize, scale=(0.8,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.1, contrast=0.1),
        ]) if is_train else transforms.Resize(resize)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        def proc(img_np):
            img = Image.fromarray(img_np.astype(np.uint8))  # grayscale HxW
            img = self.augment(img)
            x = self.to_tensor(img).repeat(3,1,1)  # to RGB 3×H×W
            return self.normalize(x)

        x1 = proc(self.X1[idx])
        x2 = proc(self.X2[idx])
        label = torch.tensor(self.y[idx], dtype=torch.float)
        return x1, x2, label

# ==================== 4. Siamese ResNet50 ====================
class SiameseResNet50(nn.Module):
    def __init__(self, drop1, drop2):
        super().__init__()
        backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        children = list(backbone.children())
        self.initial = nn.Sequential(*children[:4])  # conv1 + bn1 + relu + maxpool
        self.layer1 = children[4]
        self.layer2 = children[5]
        self.layer3 = children[6]
        self.layer4 = children[7]
        self.avgpool = children[8]
        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(drop1)
        self.dropout2 = nn.Dropout(drop2)
        self.classifier = nn.Sequential(
            nn.Linear(2048*2, 512),
            nn.ReLU(),
            self.dropout1,
            nn.Linear(512, 128),
            nn.ReLU(),
            self.dropout2,
            nn.Linear(128, 1)
        )

    def encode(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        return self.flatten(x)

    def forward(self, x1, x2):
        f1 = self.encode(x1)
        f2 = self.encode(x2)
        x = torch.cat([f1, f2], dim=1)
        return self.classifier(x).squeeze(1)

# ==================== 5. Training ====================
def train_and_evaluate(cfg):
    # Setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.manual_seed(cfg["random_seed"])
    np.random.seed(cfg["random_seed"])
    if device.type == "cuda":
        torch.cuda.manual_seed_all(cfg["random_seed"])
        torch.backends.cudnn.benchmark = True

    # Load & split data
    df = pd.read_pickle(cfg["train_pkl"])
    df["label"] = (df["label"] == 1).astype(int)
    y_all = df["label"].values
    X1_all = np.stack(df["img1"].values).astype("float32")
    X2_all = np.stack(df["img2"].values).astype("float32")
    del df; gc.collect()

    # Weight balance
    pos = (y_all == 1).sum()
    neg = (y_all == 0).sum()
    pos_weight = torch.tensor([neg/pos], device=device)

    # Split
    n = len(y_all)
    idx = np.arange(n); np.random.shuffle(idx)
    split = int((1 - cfg["val_split_ratio"]) * n)
    tr_idx, val_idx = idx[:split], idx[split:]

    train_ds = RPSDataset(X1_all[tr_idx], X2_all[tr_idx], y_all[tr_idx], resize=cfg["resize"], is_train=True)
    val_ds   = RPSDataset(X1_all[val_idx], X2_all[val_idx], y_all[val_idx], resize=cfg["resize"], is_train=False)

    train_loader = DataLoader(train_ds, batch_size=cfg["batch_size"], shuffle=True,
                              num_workers=cfg["num_workers"], pin_memory=cfg["pin_memory"], drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=cfg["batch_size"], shuffle=False,
                            num_workers=cfg["num_workers"], pin_memory=cfg["pin_memory"])

    # Model
    model = SiameseResNet50(cfg["dropout_1"], cfg["dropout_2"]).to(device)
    optimizer = optim.AdamW([
        {"params": model.initial.parameters()},
        {"params": model.layer1.parameters()},
        {"params": model.layer2.parameters()},
        {"params": model.layer3.parameters()},
        {"params": model.layer4.parameters()},
        {"params": model.classifier.parameters(), "lr": cfg["lr_classifier"]},
    ], lr=cfg["lr_backbone"], weight_decay=cfg["weight_decay"])
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=cfg["scheduler_T_0"], T_mult=cfg["scheduler_T_mult"],
        eta_min=cfg["scheduler_eta_min"]
    )
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    scaler = GradScaler()

    # Training
    best_f1, best_thr = 0.0, 0.5
    for epoch in range(1, cfg["epochs"]+1):
        model.train(); total_loss = 0.0
        for x1, x2, yb in train_loader:
            x1, x2, yb = x1.to(device), x2.to(device), yb.to(device)
            optimizer.zero_grad()
            with autocast(device_type=device.type, enabled=(device.type=="cuda")):
                logits = model(x1, x2)
                loss = criterion(logits, yb)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        train_loss = total_loss / len(train_loader)

        # Validation
        model.eval(); all_probs, all_labels = [], []
        with torch.no_grad():
            for x1, x2, yb in val_loader:
                x1, x2 = x1.to(device), x2.to(device)
                with autocast(device_type=device.type, enabled=(device.type=="cuda")):
                    logits = model(x1, x2)
                probs = torch.sigmoid(logits).cpu().numpy()
                all_probs.append(probs)
                all_labels.append(yb.numpy())

        all_probs = np.concatenate(all_probs)
        all_labels = np.concatenate(all_labels).astype(int)

        cur_thr, cur_f1 = 0.5, 0.0
        for t in np.linspace(0.1, 0.9, 81):
            p = (all_probs > t).astype(int)
            f = f1_score(all_labels, p)
            if f > cur_f1:
                cur_f1, cur_thr = f, t
        cur_acc = accuracy_score(all_labels, (all_probs > cur_thr).astype(int))
        cur_auc = roc_auc_score(all_labels, all_probs)

        print(f"Epoch {epoch:3d}/{cfg['epochs']} | TrainLoss={train_loss:.4f} "
              f"ValF1={cur_f1:.4f}@{cur_thr:.2f} | ValAcc={cur_acc:.4f} ValAUC={cur_auc:.4f}")

        scheduler.step()

        # Save best
        if cur_f1 > best_f1:
            best_f1, best_thr = cur_f1, cur_thr
            torch.save(model.state_dict(), "siamese_resnet50_best.pt")
            print(f"  ✅ New best model at epoch {epoch}! Saved to 'siamese_resnet50_best.pt'")

    print(f"\n🎯 Training complete. Best Val F1 = {best_f1:.4f} @ Thr = {best_thr:.2f}")
    return model

# ==================== 6. Run ====================
if __name__ == "__main__":
    train_and_evaluate(CONFIG)

Epoch   1/50 | TrainLoss=0.4897 ValF1=0.8418@0.46 | ValAcc=0.8305 ValAUC=0.9186
  ✅ New best model at epoch 1! Saved to 'siamese_resnet50_best.pt'
Epoch   2/50 | TrainLoss=0.3370 ValF1=0.8713@0.38 | ValAcc=0.8632 ValAUC=0.9470
  ✅ New best model at epoch 2! Saved to 'siamese_resnet50_best.pt'
Epoch   3/50 | TrainLoss=0.2803 ValF1=0.8876@0.49 | ValAcc=0.8838 ValAUC=0.9591
  ✅ New best model at epoch 3! Saved to 'siamese_resnet50_best.pt'
Epoch   4/50 | TrainLoss=0.2294 ValF1=0.8924@0.42 | ValAcc=0.8876 ValAUC=0.9592
  ✅ New best model at epoch 4! Saved to 'siamese_resnet50_best.pt'
Epoch   5/50 | TrainLoss=0.1905 ValF1=0.8928@0.50 | ValAcc=0.8902 ValAUC=0.9620
  ✅ New best model at epoch 5! Saved to 'siamese_resnet50_best.pt'
Epoch   6/50 | TrainLoss=0.1472 ValF1=0.8949@0.31 | ValAcc=0.8904 ValAUC=0.9635
  ✅ New best model at epoch 6! Saved to 'siamese_resnet50_best.pt'
Epoch   7/50 | TrainLoss=0.1200 ValF1=0.8960@0.30 | ValAcc=0.8912 ValAUC=0.9646
  ✅ New best model at epoch 7! Saved t

In [4]:
import os, gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast
from PIL import Image

# ---------------- Config ----------------
CONFIG = {
    "batch_size":     64,
    "resize":         (96, 96),
    "num_workers":    2,
    "pin_memory":     True,
    "dropout_1":      0.3,
    "dropout_2":      0.15,
    "resize":         (96, 96),
}

# ---------------- Dataset ----------------
class RPSDataset(Dataset):
    def __init__(self, X1, X2, y, resize):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.resize = resize
        self.normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        self.to_tensor = transforms.ToTensor()
        self.resize_tf = transforms.Resize(resize)

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, idx):
        def proc(img_np):
            img = Image.fromarray(img_np.astype(np.uint8))  # HxW
            img = self.resize_tf(img)
            x = self.to_tensor(img).repeat(3,1,1)
            return self.normalize(x)

        x1 = proc(self.X1[idx])
        x2 = proc(self.X2[idx])
        return x1, x2

# ---------------- Model ----------------
class SiameseResNet50(nn.Module):
    def __init__(self, drop1, drop2):
        super().__init__()
        backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        children = list(backbone.children())
        self.initial = nn.Sequential(*children[:4])
        self.layer1 = children[4]
        self.layer2 = children[5]
        self.layer3 = children[6]
        self.layer4 = children[7]
        self.avgpool = children[8]
        self.flatten = nn.Flatten()

        self.dropout1 = nn.Dropout(drop1)
        self.dropout2 = nn.Dropout(drop2)
        self.classifier = nn.Sequential(
            nn.Linear(2048 * 2, 512),
            nn.ReLU(),
            self.dropout1,
            nn.Linear(512, 128),
            nn.ReLU(),
            self.dropout2,
            nn.Linear(128, 1)
        )

    def encode(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        return self.flatten(x)

    def forward(self, x1, x2):
        f1 = self.encode(x1)
        f2 = self.encode(x2)
        x = torch.cat([f1, f2], dim=1)
        return self.classifier(x).squeeze(1)

# ---------------- Inference ----------------
def predict_test_data(cfg, model_path, test_pkl_path, output_csv_path="submission.csv"):
    if not os.path.exists(model_path):
        print(f"❌ Model file not found: {model_path}")
        return
    if not os.path.exists(test_pkl_path):
        print(f"❌ Test file not found: {test_pkl_path}")
        return

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Rebuild and load model weights
    model = SiameseResNet50(cfg["dropout_1"], cfg["dropout_2"])
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"✅ Model loaded from {model_path}")

    # Load test.pkl
    df = pd.read_pickle(test_pkl_path)
    if "img1" not in df.columns or "img2" not in df.columns:
        print("❌ test.pkl must contain 'img1' and 'img2' columns.")
        return
    if "id" not in df.columns:
        df["id"] = np.arange(len(df))

    X1 = np.stack(df["img1"].values).astype("float32")
    X2 = np.stack(df["img2"].values).astype("float32")

    dataset = RPSDataset(X1, X2, y=None, resize=cfg["resize"])
    loader = DataLoader(dataset, batch_size=cfg["batch_size"],
                        shuffle=False, num_workers=cfg["num_workers"],
                        pin_memory=cfg["pin_memory"])

    # Predict
    all_probs = []
    with torch.no_grad():
        for x1, x2 in loader:
            x1, x2 = x1.to(device), x2.to(device)
            with autocast(device_type=device.type, enabled=(device.type == "cuda")):
                logits = model(x1, x2)
            probs = torch.sigmoid(logits).cpu().numpy()
            all_probs.append(probs)

    probs = np.concatenate(all_probs)
    preds = np.where(probs >= 0.5, 1, -1)

    # Save
    df_out = pd.DataFrame({
        "id": df["id"],
        "labels": preds
    })
    df_out.to_csv(output_csv_path, index=False)
    print(f"✅ Submission saved to {output_csv_path}")

    # Cleanup
    del model, dataset, loader, df, X1, X2
    gc.collect()
    if device.type == 'cuda':
        torch.cuda.empty_cache()

# ---------------- Run ----------------
if __name__ == "__main__":
    MODEL_FILE_PATH = "siamese_resnet50_best.pt"
    TEST_DATA_PATH = "test.pkl"
    OUTPUT_CSV_PATH = "submission_resnet50_probably_best_of_day1.csv"

    predict_test_data(CONFIG, MODEL_FILE_PATH, TEST_DATA_PATH, OUTPUT_CSV_PATH)

Using device: cuda
✅ Model loaded from siamese_resnet50_best.pt
✅ Submission saved to submission_resnet50_probably_best_of_day1.csv
